In [1]:
import os
import glob
import nibabel as nib
import numpy as np

In [ ]:
def apply_brain_mask(nifti_file_path, mask_file_path, threshold, output_directory):
    # Load NIfTI file and mask
    nifti_img = nib.load(nifti_file_path)
    mask_img = nib.load(mask_file_path)
    
    data = nifti_img.get_fdata()
    mask = mask_img.get_fdata() > threshold

    # Apply the mask to each time point
    masked_data = data.copy()
    for t in range(data.shape[-1]):
        masked_data[..., t] = np.where(mask, data[..., t], 0)

    # Save the masked data to a new NIfTI file
    masked_nifti = nib.Nifti1Image(masked_data, nifti_img.affine)
    output_path = os.path.join(output_directory, os.path.basename(nifti_file_path).replace('.nii', '_masked.nii'))
    #nib.save(masked_nifti, output_path)
    
    return masked_nifti
    #return output_path


In [2]:
def normalize_and_save(nifti_file_path, nifti_img, output_directory):
    # Load the NIfTI file data
    data = nifti_img.get_fdata()

    # Calculate the mean and standard deviation along the time dimension
    means = np.mean(data, axis=3)
    stds = np.std(data, axis=3)

    # Avoid division by zero by adding a small constant to stds
    stds[stds == 0] = 1e-10

    # Normalize the data
    normalized_data = (data - means[..., np.newaxis]) / stds[..., np.newaxis]

    # Save the normalized data to a new NIfTI file
    normalized_nifti = nib.Nifti1Image(normalized_data, nifti_img.affine)
    output_file_name = os.path.basename(nifti_file_path).replace('.nii', '_quirkynormalized.nii')
    output_path = os.path.join(output_directory, output_file_name)
    nib.save(normalized_nifti, output_path)

    return output_path, means, stds

In [4]:
base_dir= "/home/fgomezacebo/lood_storage/divi/Projects/fMRS/students/Fernando/Renamed_scans_conditions"
condition = "high"  # Change this to 'high' or 'placebo' as needed
search_pattern = os.path.join(base_dir, "sub-0**/ses-{}/func/*.nii.gz".format(condition))
directory_path = "/home/fgomezacebo/scratch"
mask_file_path = '/home/fgomezacebo/scratch/MNI152lin_T1_2mm_brain_pve_GM.nii.gz'

for file_path in glob.glob(search_pattern):
        # Load the NIfTI file
        nifti_img = apply_brain_mask(file_path, mask_file_path, 0.3, directory_path)
        norm_nifti = normalize_and_save(file_path, nifti_img, directory_path)
